### Crop the images

In [1]:
import os
from osgeo import gdal, ogr
from shapely.geometry import box
import geopandas as gpd

In [7]:
tif_path = '../data/actual_data/2023-02-23-bakonyszucs_im_cropped.tif'
shp_path = '../data/actual_data/Pont.shp'
out_folder ='../data/distributed'

### split tiff

In [39]:
import os
from osgeo import gdal
from shapely.geometry import box, Point
import geopandas as gpd

def split_tiff(tif_path, out_folder, tile_size=(250, 250)):
    ds = gdal.Open(tif_path)
    if ds is None:
        print(f"Could not open input TIFF file: {tif_path}")
        return
    
    #get raster width and height
    width = ds.RasterXSize
    height = ds.RasterYSize

    #calc. the number of rows and columns
    num_cols = (width + tile_size[0] - 1) // tile_size[0]
    num_rows = (height + tile_size[1] - 1) // tile_size[1]
    
    print('start splitting tiffs . . .')
    for i in range(num_rows):
        for j in range(num_cols):
            #bbox coordinates
            xmin = j * tile_size[0]
            ymin = i * tile_size[1]
            xmax = min((j + 1) * tile_size[0], width)
            ymax = min((i + 1) * tile_size[1], height)

            #crop & save
            output_tif = os.path.join(out_folder, f'tile_tiff_{i}_{j}.tif')
            gdal.Translate(output_tif, ds, srcWin=[xmin, ymin, xmax - xmin, ymax - ymin])
    print('splitted successfully')
    
out_folder = '../data/tiles'
tile_size = (250, 250)  #tile size in pixels
split_tiff(tif_path, out_folder, tile_size)


start splitting tiffs . . .
splitted successfully


### splitting shapefile with splitted tiff file's locations

In [38]:
import geopandas as gpd
import rasterio
from shapely.geometry import box
from rasterio.mask import mask
import os

def split_shp(bigger_shapefile_path, tiff_file_path, output_folder):
    #read the shp
    gdf_bigger = gpd.read_file(bigger_shapefile_path)
    
    #read the tiff and get the bbox
    with rasterio.open(tiff_file_path) as src:
        bbox = box(*src.bounds)
    
    #clip the bigger shapefile to the extent of the TIFF file
    gdf_intersection = gdf_bigger[gdf_bigger.intersects(bbox)]
    
    splitted_name = os.path.basename(tiff_file_path).split('_')
    out_path = os.path.join(output_folder,
                            f'tile_shp_{splitted_name[2]}_{os.path.splitext(splitted_name[3])[0]}.shp')
    gdf_intersection.to_file(out_path)

tif_path = '../data/actual_data/2023-02-23-bakonyszucs_im_cropped.tif'
shp_path = '../data/actual_data/Pont.shp'
out_folder ='../data/distributed'
split_shp(shp_path, 'output_tiles/tile_tiff_0_0.tif', '../data/tiles')


## concat tiff & shp split

In [48]:
import os
def split(tiff_path, shp_path, output_folder, tile_size=(250, 250)):
    split_tiff(tiff_path,output_folder,tile_size)

    print('get available tiles')
    extension = '.tif'
    tiff_files=[]
    for tiff_file in os.listdir(output_folder):
        if tiff_file.endswith(extension):
            tiff_files.append(os.path.join(output_folder, tiff_file))
            
    print('split shps')
    for tiff_path in tiff_files:
        split_shp(shp_path, tiff_path, output_folder)
    print('success')

    

In [49]:
split(tif_path,shp_path,'../data/tiles')

start splitting tiffs . . .
splitted successfully
get available tiles
split shps
success


/Users/banoczymartin/OE Projects/Mapping Upper Canopy Level/venvs_can/canopysegmentation/lib/python3.9/site-packages/geopandas/io/file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [27]:
def get_tiff_size(tiff_file_path):
    with rasterio.open(tiff_file_path) as src:
        width = src.width
        height = src.height
    return width, height
print(get_tiff_size('output_tiles/tile_tiff_2_4.tif'))
get_tiff_size('output_tiles/tile_tiff_r_2_4.tif')

(197, 233)


(127, 194)

### Convert SHP to PNG

In [58]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt

def convert_SHPtoPNG(shp_path,png_path, point_size=1, dpi = 300,bg_color='black',fg_color='white', fig_size=(50,50)):
    '''
    returns the point as well as a list of tuples
    '''
    
    gdf = gpd.read_file(shp_path)

    xmin, ymin, xmax, ymax = gdf.total_bounds

    #plot points as white boxes on a black background
    fig, ax = plt.subplots(figsize=fig_size)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_facecolor(bg_color)
    ax.scatter(gdf.geometry.x, gdf.geometry.y, s=point_size, color=fg_color) #point size

    #save the plot as a PNG image
    plt.savefig(png_path, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=dpi)
    plt.close()
    
    #points = [(p.x, p.y) for p in gdf.geometry]
    return list(zip(gdf.geometry.x,gdf.geometry.y))#points (X, Y)
    
shp_path = '../data/tiles/tile_shp_0_0.shp'
out_path = '../data/actual_data/out.png'
points = convert_SHPtoPNG(shp_path,out_path,point_size=5, fig_size=(10,10))
print(points)

[(549243.3870318223, 221151.71778455636), (549253.7454693333, 221151.77731580642), (549253.74174863, 221174.52941543964), (549248.979248625, 221167.50472793222), (549242.1331548678, 221161.84925917626), (549255.7062798821, 221157.97972792215), (549259.0995611356, 221169.05254043386), (549256.0039361324, 221158.03925917222), (549250.7815254254, 221162.48085515414), (549251.6184003273, 221160.51942960278), (549246.910979004, 221160.38866789936), (549257.5549816627, 221157.17192999512), (549244.0603738694, 221155.91661764227), (549238.11888897, 221155.47693141436)]


In [35]:
gdf = gpd.read_file(shp_path)
min_x, min_y, max_x, max_y = gdf.total_bounds

# Calculate the difference between maximum and minimum coordinate values
x_resolution = max_x - min_x
y_resolution = max_y - min_y
print(x_resolution, y_resolution)

177.4473663936369 109.0244564920722


** copy shp params **

In [34]:
output_shp = 'output_shapefile.shp' #'../data/actual_data/output_points.shp'
input_shp = '../data/actual_data/output_points.shp'
input_ds = ogr.Open(shp_path)

if input_ds is None:
    print(f"Could not open input shapefile: {shp_path}")
    exit()
    
# Get the input layer
input_layer = input_ds.GetLayer()

# Create a new shapefile for writing
driver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(output_shp):
    driver.DeleteDataSource(output_shp)
output_ds = driver.CreateDataSource(output_shp)
output_layer = output_ds.CreateLayer(input_layer.GetName(), input_layer.GetSpatialRef(), input_layer.GetGeomType())

# Define the fields and add them to the output layer
field_def_list = []
for i in range(input_layer.GetLayerDefn().GetFieldCount()):
    field_def = input_layer.GetLayerDefn().GetFieldDefn(i)
    field_def_list.append(field_def)
    output_layer.CreateField(field_def)
print(field_def_list, len(field_def_list))
# Copy features from input layer to output layer with modified geometry (e.g., move features)
for feature in input_layer:
    geom = feature.GetGeometryRef()
    # Modify the geometry here (e.g., move the feature)
    # Example: geom.SetPoint(0, NEW_X, NEW_Y)  # Move the first point of the geometry
    new_feature = ogr.Feature(output_layer.GetLayerDefn())
    new_feature.SetGeometry(geom)
    if geom is not None:
        print(geom.GetPoint())
    for i, field_def in enumerate(field_def_list):
        new_feature.SetField(i, feature.GetField(i))
    output_layer.CreateFeature(new_feature)
    new_feature = None

# Close the shapefiles
input_ds = None
output_ds = None

[<osgeo.ogr.FieldDefn; proxy of <Swig Object of type 'OGRFieldDefnShadow *' at 0x15bab1c00> >] 1
(549313.53553259, 221100.371878735, 0.0)
(549312.6498544463, 221111.39365119016, 0.0)
(549294.9854959132, 221106.03037798652, 0.0)
(549304.5311381288, 221122.12019759743, 0.0)
(549320.7931729344, 221098.354500741, 0.0)
(549323.6962290722, 221103.3733435554, 0.0)
(549332.4053974855, 221097.91166166915, 0.0)
(549333.586301677, 221104.11140867518, 0.0)
(549361.140732815, 221102.09403068115, 0.0)
(549347.51113027, 221118.67589370522, 0.0)
(549347.806356318, 221111.34444684887, 0.0)
(549340.1796834137, 221105.58753891473, 0.0)
(549349.528508264, 221104.55424774703, 0.0)
(549342.6399004796, 221093.97531436372, 0.0)
(549363.010497785, 221116.31408532197, 0.0)
(549308.8365179942, 221128.66437499272, 0.0)
(549335.1608405992, 221133.43719610054, 0.0)
(549339.7368443417, 221128.9103966993, 0.0)
(549327.8293937428, 221127.87710553163, 0.0)
(549295.1577111079, 221113.75545957344, 0.0)
(549299.044854072,

### get the white dots center point on a png image

In [55]:
image_path = '../data/actual_data/out.png'

In [62]:
#very slow compared to opencv
from PIL import Image
import numpy as np
from scipy import ndimage
def getPoints_fromPNG(image_path):
    img = Image.open(image_path)
    
    #grayscale
    img_gray = img.convert('L')
    
    threshold = 254
    img_binary = np.array(img_gray) > threshold
    
    #find connected components in the binary mask
    labeled_img, num_features = ndimage.label(img_binary)
    
    white_dot_centers = []
    
    for label in range(1, num_features + 1):
        #find coordinates of all pixels belonging to the connected component
        rows, cols = np.where(labeled_img == label)
        
        #calc. centroid as the mean of x and y coordinates
        center_x = np.mean(cols)
        center_y = np.mean(rows)
        
        #append centroid
        white_dot_centers.append((center_x, center_y))
    
    return white_dot_centers

white_dot_centers = getPoints_fromPNG(image_path)
print(white_dot_centers)
white_dot_centers.count((1071.0,3.0))
len(white_dot_centers)

[(1813.0, 49.06666666666667), (2402.9333333333334, 600.0), (1286.0, 757.0), (1485.0, 1265.0), (527.0, 1329.0), (1578.0, 1464.0), (1056.0, 1477.0), (2064.0, 1715.0), (2031.0, 1721.0), (2236.0, 1803.0), (741.0, 1930.0), (86.06666666666666, 1975.0), (1814.0, 2348.127450980392), (666.0, 2350.9333333333334)]


14

In [60]:
#fastest way
import cv2
import numpy as np
def getPoints_fromPNG(image_path):
    img = cv2.imread(image_path)
    
    #grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #threshold
    _, img_binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
    
    #find contours
    contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    white_dot_centers = []
    for contour in contours:
        #calculate the moments
        M = cv2.moments(contour)
        
        #calculate centroid coordinates
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            white_dot_centers.append((cX, cY))
    
    return white_dot_centers


white_dot_centers = getPoints_fromPNG(image_path)
print(white_dot_centers)
white_dot_centers.count((1071, 3))
len(white_dot_centers)

[(666, 2351), (1814, 2348), (85, 1975), (741, 1930), (2236, 1803), (2031, 1721), (2064, 1715), (1056, 1477), (1578, 1464), (527, 1329), (1485, 1265), (1286, 757), (2403, 600), (1813, 48)]


14

### Create shp from points

In [66]:
import os
from osgeo import ogr, osr

def create_shapefile(output_shp, points):
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(output_shp):
        driver.DeleteDataSource(output_shp)
    output_ds = driver.CreateDataSource(output_shp)
    
    spatial_ref = osr.SpatialReference()
    spatial_ref.ImportFromEPSG(4326)  # WGS84
    
    #new layer
    output_layer = output_ds.CreateLayer("points", spatial_ref, ogr.wkbPoint)
    
    #define a field for the point ID
    id_field = ogr.FieldDefn("ID", ogr.OFTInteger)
    output_layer.CreateField(id_field)
    
    #create points and add them to the layer
    for i, (x, y) in enumerate(points):
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(y,x)
        
        feature = ogr.Feature(output_layer.GetLayerDefn())
        feature.SetGeometry(point)
        feature.SetField("ID", i+1)
        output_layer.CreateFeature(feature)
        
        feature = None
    
    output_ds = None


output_shp = 'output_shapefile2.shp'
points = white_dot_centers#[(10, 20), (30, 40), (50, 60)]  # Example list of coordinates

create_shapefile(output_shp, points)


## output shapefiles are in wrong direction and mislocated.